# Dataset Description
## train.csv: A full training dataset with the following attributes:

1. #### id: unique id for a news article
2. #### title: the title of a news article
3. #### author: author of the news article
4. #### text: the text of the article; could be incomplete
5. #### label: a label that marks the article as potentially unreliable
###### 1: unreliable
###### 0: reliable

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# printing the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data Collection and Pre processing

In [4]:
data=pd.read_csv("/kaggle/input/fake-news/train.csv")
test_data=pd.read_csv("/kaggle/input/fake-news/test.csv")
test_data_id=test_data['id']
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
test_data.shape

(5200, 4)

In [6]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
test_data.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [8]:
# replacing the null values in dataset with empty string.
data.fillna('', inplace=True)
test_data.fillna('', inplace=True)

In [9]:
# merging the title and author column 
data['content']=data['author']+' '+data['title']
test_data['content']=test_data['author']+' '+test_data['title']
data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


# Stemming

In [10]:
port_stem=PorterStemmer()

In [11]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [12]:
data['content']=data['content'].apply(stemming)
test_data['content']=test_data['content'].apply(stemming)

In [13]:
test_data['content']

0       david streitfeld specter trump loosen tongu pu...
1       russian warship readi strike terrorist near al...
2       common dream nodapl nativ american leader vow ...
3       daniel victor tim tebow attempt anoth comeback...
4        truth broadcast network keiser report meme war e
                              ...                        
5195    jodi rosen bangladeshi traffic jam never end n...
5196    sheryl gay stolberg john kasich sign one abort...
5197    mike mcphate california today exactli sushi ne...
5198                us marin deploy russian border norway
5199        teddi wayn awkward sex onscreen new york time
Name: content, Length: 5200, dtype: object

In [14]:
# seperating target variable and independent variables.

X=data['content'].values
y=data['label'].values

In [15]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [16]:
X_test_val=test_data['content'].values

In [17]:
# converting textual data to numerical data for test
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X)
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210687 stored elements and shape (20800, 17128)>
  Coords	Values
  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20

In [18]:
# converting textual data to numerical data
X_test_val=vectorizer.transform(X_test_val)
print(X_test_val)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 50660 stored elements and shape (5200, 17128)>
  Coords	Values
  (0, 16996)	0.08864595251126647
  (0, 16088)	0.3205594987716609
  (0, 15582)	0.11032513315099028
  (0, 15368)	0.38547947461491155
  (0, 15295)	0.08697876504498342
  (0, 14605)	0.3970837204511862
  (0, 14591)	0.34467447913058513
  (0, 13850)	0.32956045112215354
  (0, 12030)	0.38547947461491155
  (0, 10306)	0.08568694729690247
  (0, 8842)	0.36912420870480717
  (0, 3623)	0.209228842739656
  (1, 16473)	0.4921981982038152
  (1, 15142)	0.3550835494981517
  (1, 14603)	0.35559908473568963
  (1, 13048)	0.3052969733971102
  (1, 12300)	0.3838422559012773
  (1, 10219)	0.37818137822242154
  (1, 347)	0.3487744590431239
  (2, 16746)	0.3016628915147435
  (2, 16354)	0.2909537422615255
  (2, 14457)	0.30637021801826614
  (2, 11598)	0.21447077225895875
  (2, 10419)	0.35982822390508096
  (2, 10176)	0.3194727535249622
  :	:
  (5196, 5884)	0.2500401085058591
  (5196, 1517)	0.199403969

In [19]:
X.shape

(20800, 17128)

In [20]:
X_test_val.shape

(5200, 17128)

# Splitting data into train test

In [21]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

In [22]:
print(X.shape, X_train.shape, X_test.shape)

(20800, 17128) (16640, 17128) (4160, 17128)


In [23]:
model=LogisticRegression()

In [24]:
model.fit(X_train, y_train)

LogisticRegression()

In [25]:
y_pred=model.predict(X_test)
y_pred

array([1, 0, 1, ..., 1, 1, 0])

# Accuracy Score

In [26]:
accuracy=accuracy_score(y_pred, y_test)
print("Accuracy on test data: ", accuracy)

Accuracy on test data:  0.9790865384615385


# Building a predictive system

In [27]:
X_test[0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 13 stored elements and shape (1, 17128)>

In [28]:
input_data=X_test[0]

prediction=model.predict(input_data)

if prediction[0]==0:
    print("Real news")
    
else:
    print("Fake news")

Fake news


In [29]:
y_test[0]

1

In [30]:
y_pred_test=model.predict(X_test_val)
y_pred_test.shape

(5200,)

In [31]:
test_data_id=pd.DataFrame(test_data_id)
test_data_id

,id
0,20800
1,20801
2,20802
3,20803
4,20804
...,...
5195,25995
5196,25996
5197,25997
5198,25998


In [32]:
y_pred_res=pd.DataFrame(y_pred_test)
y_pred_res

,0
0,0
1,1
2,1
3,0
4,1
...,...
5195,0
5196,0
5197,0
5198,1


In [33]:
# y_pred_res.columns = y_pred_res.columns.str.replace('0', 'label')
y_pred_res = y_pred_res.rename(columns={0: 'label'})
y_pred_res

,label
0,0
1,1
2,1
3,0
4,1
...,...
5195,0
5196,0
5197,0
5198,1


In [34]:
# result=test_data_id.append(y_pred_res)
result=pd.concat([test_data_id, y_pred_res], axis=1)

result

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1
...,...,...
5195,25995,0
5196,25996,0
5197,25997,0
5198,25998,1


In [35]:
# Export to CSV
result.to_csv('my_data.csv', index=False)